In [1]:
import json
import requests
from src import utils
import time
import os
from pprint import pprint
import random

In [ ]:
# peft tunning
# peft1 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft1-test.jsonl'
# peft2 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft2-test.jsonl'

# ICL
zs = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-public_mwp_data_v2_preprocess'
zs_cot = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-cot-public_mwp_data_v2_preprocess'
# one_shot_cot = '/userhomes/philhoon/kt-ai-challenge/result/one-shot-cot-result-ko-gsm8k-test.jsonl'


In [2]:
## Test dataset

# GSM8K 8792 (7473/1319)
gsm8k = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

# MWP_KR_DATA 3750
mwp_kr = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning-test.jsonl'

# AddSub 395
addsub = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_addsub-test.json'

# MultiArith 600
multiarith = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_multiarith-test.json'

# SingleEq 395
single_eq = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_single_eq-test.json'

In [8]:
gsm8k_data = utils.read_jsonlines(gsm8k_train)
mwp_kr_data = utils.read_jsonlines(mwp_kr)
gsm8k_data = utils.read_jsonlines(gsm8k_train)
gsm8k_data = utils.read_jsonlines(gsm8k_train)
gsm8k_data = utils.read_jsonlines(gsm8k_train)

In [4]:
print(len(gsm8k_train))
print(len(mwp_kr))
print(len(addsub))
print(len(multiarith))
print(len(single_eq))

96
96


    Zero-shot
        ACC : 3/50
    Zero-shot-CoT
        ACC : 2/50
    One-shot-CoT
        ACC : 1/50
    PEFT1 (fine-tuning)
        ACC : 2/50
    PEFT2 (few-shot fine-tunning)
        ACC : 3/50

In [ ]:
# d_p1 = utils.read_jsonlines(peft1)
# d_p2 = utils.read_jsonlines(peft2)

d_zs = utils.read_jsonlines(zs)
d_zsc = utils.read_jsonlines(zs_cot)
# d_osc = utils.read_jsonlines(one_shot_cot)

In [ ]:
# print(len(d_p1))
# print(len(d_p2))
print(len(d_zs))
print(len(d_zsc))
# print(len(d_osc))

In [ ]:
random.seed(42)

random_indices = random.sample(range(len(d_zs)), 50)

In [ ]:
# random_indices

In [ ]:
d_zs[2619]

In [ ]:
data = d_zs

In [ ]:
for idx in random_indices:
    ins = data[idx]
#     pprint(ins)

    print(f"question\n{ins['question']}")
    print(f"zs_infer\n{ins['zs_infer']}")
    print('---------------------------------')
    print(f"answer\n{ins['answer']}")
    print('=================================')

In [ ]:
for ins in d_osc:
    print(f"question : {}")
    break

In [ ]:
zs_cot_data = utils.read_jsonlines(zs_cot)
zs_data = utils.read_jsonlines(zs)

In [ ]:
print(len(zs_cot_data))
len(zs_cot_data) == len(zs_data)

In [ ]:
from pprint import pprint

In [ ]:
cnt = 0
for ins in zs_data:
#     pprint(ins)
    print(f"question :\n{ins['question']}")
#     print(f"zs_cot_infer :\n{ins['zs_cot_infer']}")
    print(f"zs_infer :\n{ins['zs_infer']}")
    print(f"answer :\n{ins['answer']}")
    print("===========================")
    cnt += 1
    if cnt == 50:
        break